# Gene x Noise interaction analysis

Using the recoding gene level dosage to do Gene x Noise interaction analysis

In [1]:
# Common variables Columbia's cluster
UKBB_PATH=$HOME/UKBiobank
UKBB_yale=$HOME/UKBiobank_Yale_transfer
USER_PATH=$HOME/working
container_lmm=$HOME/containers/lmm.sif
container_marp=$HOME/containers/marp.sif
container_annovar=$HOME/containers/gatk4-annovar.sif
hearing_pheno_path=$UKBB_PATH/phenotype_files/hearing_impairment
tpl_file=$USER_PATH/bioworkflows/admin/csg.yml
formatFile_fastgwa=$USER_PATH/UKBB_GWAS_dev/data/fastGWA_template.yml
formatFile_bolt=$USER_PATH/UKBB_GWAS_dev/data/boltlmm_template.yml
formatFile_saige=$USER_PATH/UKBB_GWAS_dev/data/saige_template.yml
formatFile_regenie=$USER_PATH/UKBB_GWAS_dev/data/regenie_template.yml
# Workflows
lmm_sos=$USER_PATH/bioworkflows/GWAS/LMM.ipynb
anno_sos=$USER_PATH/bioworkflows/variant-annotation/annovar.ipynb
clumping_sos=$USER_PATH/bioworkflows/GWAS/LD_Clumping.ipynb
extract_sos=$USER_PATH/bioworkflows/GWAS/Region_Extraction.ipynb
snptogene_sos=$USER_PATH/UKBB_GWAS_dev/workflow/snptogene.ipynb

# LMM directories for array data data
lmm_imp_dir_fastgwa=$UKBB_PATH/results/FastGWA_results/results_array_data
lmm_imp_dir_bolt=$UKBB_PATH/results/BOLTLMM_results/results_array_data
lmm_imp_dir_saige=$UKBB_PATH/results/SAIGE_results/results_array_data
lmm_imp_dir_regenie=$UKBB_PATH/results/REGENIE_results/results_array_data

# LMM directories for exome data
lmm_exome_dir_fastgwa=$UKBB_PATH/results/FastGWA_results/results_array_data
lmm_exome_dir_bolt=$UKBB_PATH/results/BOLTLMM_results/results_array_data
lmm_exome_dir_saige=$UKBB_PATH/results/SAIGE_results/results_array_data
lmm_exome_dir_regenie=$UKBB_PATH/results/REGENIE_results/results_array_data
### Specific to REGENIE
bsize=1000
lowmem=~/project/guangyou/interaction/scratch60/
lowmem_dir=~/project/guangyou/interaction/scratch60/predictions
trait=bt
minMAC=4
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
reverse_log_p=True
numThreads=2

Traceback (most recent call last):
  File "/home/gl2776/miniconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/gl2776/miniconda3/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/gl2776/miniconda3/lib/python3.8/site-packages/bash_kernel/__main__.py", line 1, in <module>
    from ipykernel.kernelapp import IPKernelApp
  File "/home/gl2776/miniconda3/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 42, in <module>
    from .ipkernel import IPythonKernel
  File "/home/gl2776/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 18, in <module>
    from .zmqshell import ZMQInteractiveShell
  File "/home/gl2776/miniconda3/lib/python3.8/site-packages/ipykernel/zmqshell.py", line 43, in <module>
    from ipykernel.displayhook import ZMQShellDisplayHook
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>",

In [2]:
## All filters set to 0 because the version of the bfile has already been QC'ed previously and there is not need to do it here
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
lmm_dir_regenie=/home/gl2776/project/guangyou/epistasis/20220307_regenie_interaction_GenexNoise/20220307_f3393
lmm_sbatch_regenie=$lmm_dir_regenie/f3393_noise_interaction-regenie_$(date +"%Y-%m-%d").sbatch
#Use the phenotype file for the 500K individuals
phenoFile=~/UKBiobank/results/REGENIE_results/results_imputed_data/2022_02_15_regenie_interaction_noise/phenotypes/021522_UKBB_Hearing_aid_f3393_expandedwhite_15601cases_237318ctrl_500k_PC1_PC2_noise
phenoCol=f3393
covarCol=sex
qCovarCol="age PC1 PC2"
#Use the original bed files that passed QC using Megan's parameters geno=0.01, mind=0.1, maf=0.01, hwe=5e-08
bfile=$UKBB_PATH/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
# Use the gene bfile here for the association
genoFile="~/project/guangyou/epistasis/chr1/ukb23155_c1_b0_v1.filtered.gene.bed"
for i in {2..22}
do
genoFile=`echo "${genoFile} ~/project/guangyou/epistasis/chr${i}/ukb23155_c${i}_b0_v1.filtered.gene.bed"`
done
## After doing the annotation you can create the annotatted manhatan plot
lmm_job_size=20
bgenMinMAF=0.001
bgenMinINFO=0.8
regenie_sos=~/working/interaction/newv/Regenie_interaction_GxE.ipynb
regenieFile=/mnt/mfs/statgen/guangyou/containers/regenie.v.2.3
lowmem_dir=$lmm_dir_regenie/scratch60/
covariates="f3393_f4825_1 f3393_f4836_1 f3393_f4825_f4836_1 f3393_f4825_2 f3393_f4836_2 f3393_f4825_f4836_2"
# If --annotate then it will add the label to the plot otherwise --no-annotate

lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --regenieFile $regenieFile
    --sampleFile $sampleFile
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --lowmem_dir $lowmem_dir
    --trait $trait
    --covariates $covariates
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --minMAC $minMAC
    --job_size $lmm_job_size
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/working/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $regenie_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args" 

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/gl2776/project/guangyou/epistasis/20220307_regenie_interaction_GenexNoise/20220307_f3393/f3393_noise_interaction-regenie_2022-03-07.sbatch
INFO: Workflow csg (ID=w6b6875d2b67ec92f) is executed successfully with 1 completed step.
